In [2]:
import json
from  pathlib import Path
path = Path("../resources/ufrpe/qa.txt")
with open(path, 'r') as f:
    object_list = []
    for line in f.readlines():
        object_list.append(json.loads(line))

In [10]:
contexts = []
questions = []
answers = []

In [11]:
for i in object_list:
    questions.append(i['pergunta'])
    answers.append(i['resposta'])

In [28]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('neuralmind/bert-base-portuguese-cased')

ImportError: cannot import name 'BertForQuestionAnswering'

In [29]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [23]:
question = "Quando começam as aulas do PLE?"
answer_text = "O primeiro PLE (2020.3) iniciará no dia 17 de agosto. Confira o Calendário Acadêmico com todas as datas"
input_ids = tokenizer.encode(question, answer_text)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 42 tokens.


In [24]:
# BERT only needs the token IDs, but for the purpose of inspecting the
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):

    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')

    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
Quando        1,643
começam       8,749
as              260
aulas         6,880
do              171
P               212
##L          22,327
##E          22,309
?               136

[SEP]           102

O               231
primeiro        652
P               212
##L          22,327
##E          22,309
(               113
202          14,979
##0          22,307
.               119
3               511
)               114
iniciar       9,070
##á          22,303
no              202
dia             644
17            1,040
de              125
agosto        1,600
.               119
Conf          9,425
##ira           364
o               146
Cale         21,716
##ndário      7,566
Acadêm       18,290
##ico           319
com             170
todas         1,485
as              260
datas        10,995

[SEP]           102



In [25]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [26]:
import torch
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

NameError: name 'model' is not defined

In [ ]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

# Referencia
* https://mccormickml.com/2020/03/10/question-answering-with-a-fine-tuned-BERT/
* https://medium.com/datadriveninvestor/extending-google-bert-as-question-and-answering-model-and-chatbot-e3e7b47b721a
  * Com preparação dos dados